# Setup

In [20]:
%conda install pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.
^C

Note: you may need to restart the kernel to use updated packages.


In [21]:
%conda install pandas

Solving environment: ...working... done

## Package Plan ##

  environment location: d:\Users\zhuan\anaconda3\envs\behaviourclone

  added / updated specs:
    - pandas


The following NEW packages will be INSTALLED:

  bottleneck         pkgs/main/win-64::bottleneck-1.3.5-py39h080aedc_0
  numexpr            pkgs/main/win-64::numexpr-2.8.3-py39hb80d3ca_0
  pandas             pkgs/main/win-64::pandas-1.4.3-py39hd77b12b_0
  pytz               pkgs/main/win-64::pytz-2022.1-py39haa95532_0


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
import pandas as pd
from PIL import Image, ImageFilter

import torch
from torchvision.io import read_image
from torch.utils.data import Dataset
from torchvision import transforms, utils

# Load Data

In [ ]:
IMG_WIDTH = 224
IMG_HEIGHT = 224
LABEL_TO_NUM = {
    'none': 0,
    'w': 1,
    'a': 2,
    's': 3,
    'd': 4
}

In [ ]:
class ScreenshotsDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform

        paths = []
        labels = []
        for label in os.listdir(root_dir):
            for image in os.listdir(os.path.join(root_dir, label)):
                paths.append(os.path.join(root_dir, label, image))
                labels.append(LABEL_TO_NUM[label])
                print(os.path.join(root_dir, label, image))

        df_dict = {
            'path': paths,
            'label': labels
        }
        self.df = pd.DataFrame(df_dict)

    def __len__(self):
        return len(self.df.index)

    def __getitem__(self, idx):
        img_path = self.df.iloc[idx, 0]
        label = self.df.iloc[idx, 1]
        image = read_image(img_path)
        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
data_transforms = transforms.Compose([
    transforms.Resize(IMG_WIDTH, IMG_HEIGHT), 
    transforms.Grayscale(), 
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

data = ScreenshotsDataset(root_dir="data", transform=data_transforms)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Train Model

# Save Model